# (06) Dashboard Unsupervised

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
from datetime import datetime

role = get_execution_role()

region = boto3.Session().region_name

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# download pred proba and li

s3_merged_inferences = "s3://aai-590-tmp2/data_split/train_val/validation2/evaluation/merged_inferences.csv"
s3_label_map_uri = f"s3://aai-590-tmp2/data_split/train_val/label_mapping.json"

In [3]:
# download the label map and pred probas
merged_inferences_pd = pd.read_csv(s3_merged_inferences)
label2idx = pd.read_json(s3_label_map_uri, typ="series").to_dict()

In [9]:
idx2label = {int(label2idx[i]):i for i in label2idx.keys()}
idx2label

{0: 'car',
 1: 'coyote',
 2: 'deer',
 3: 'bobcat',
 4: 'dog',
 5: 'skunk',
 6: 'empty',
 7: 'cat',
 8: 'opossum',
 9: 'squirrel',
 10: 'raccoon',
 11: 'rodent',
 12: 'rabbit',
 13: 'bird',
 14: 'badger',
 15: 'fox',
 16: 'lizard'}

In [10]:
merged_inferences_pd.head()

,Unnamed: 0,filename,prediction,label,category_id,bbox,image_id,location,split_type,date_captured,year_month,minute_of_day,day_of_year,minOfDay_sin,minOfDay_cos,dayOfYear_sin,dayOfYear_cos,pred_label_enc,pred_label,max_pred_prob
0,0,59be3eaf-23d2-11e8-a6a3-ec086b02610b_0.jpg,"[0.011848283000290002, 0.005372683517634, 0.00...",bobcat,6,"[5.12, 122.88, 1244.16, 1359.36]",59be3eaf-23d2-11e8-a6a3-ec086b02610b,61,train,2013-02-11 19:00:57,2013-02,1140.0,42.0,-0.965926,0.258819,0.661635,0.749826,3,bobcat,0.949937
1,1,5a07cc20-23d2-11e8-a6a3-ec086b02610b_0.jpg,"[6.33745358413762e-08, 3.704282747207799e-08, ...",opossum,1,"[1763.1519775391, 862.2836303711, 220.84802246...",5a07cc20-23d2-11e8-a6a3-ec086b02610b,24,train,2012-04-27 01:57:43,2012-04,117.0,118.0,0.488621,0.872496,0.895839,-0.444378,8,opossum,0.999996
2,2,5971fb7f-23d2-11e8-a6a3-ec086b02610b_0.jpg,"[1.9673431178417192e-14, 4.767381014194246e-17...",opossum,1,"[1474.56, 632.32, 371.2, 235.52]",5971fb7f-23d2-11e8-a6a3-ec086b02610b,38,train,2012-06-11 20:22:22,2012-06,1222.0,163.0,-0.814116,0.580703,0.329408,-0.944188,8,opossum,1.000000
3,3,59edef9c-23d2-11e8-a6a3-ec086b02610b_0.jpg,"[3.447257995503605e-06, 0.000163651013281, 5.3...",opossum,1,"[307.2, 765.44, 435.2, 230.4]",59edef9c-23d2-11e8-a6a3-ec086b02610b,38,train,2012-04-16 23:24:51,2012-04,1404.0,107.0,-0.156434,0.987688,0.963471,-0.267814,8,opossum,0.964972
4,4,59849691-23d2-11e8-a6a3-ec086b02610b_2.jpg,"[0.001230833004228, 9.84221260296181e-05, 0.00...",bird,11,"[1359.36, 652.8, 69.12, 97.28]",59849691-23d2-11e8-a6a3-ec086b02610b,61,train,2013-02-15 09:32:31,2013-02,572.0,46.0,0.601815,-0.798636,0.711657,0.702527,13,bird,0.410181


## 6.1 Prediction Confidence Distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Example: Simulated model prediction confidences (probabilities for the predicted class)
# In practice, replace with: model.predict_proba(X) or model(X) for your test set
np.random.seed(0)
confidences = np.random.beta(2, 5, size=1000)  # Simulated confidence scores (0 to 1)

# Plot histogram of confidence scores
plt.figure(figsize=(8, 5))
sns.histplot(confidences, bins=20, kde=True, color="dodgerblue")
plt.title("Prediction Confidence Distribution")
plt.xlabel("Confidence Score")
plt.ylabel("Frequency")
plt.show()